In [30]:
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.regularizers import l2
from keras.layers import Conv2D,Dense,MaxPooling2D,Flatten,BatchNormalization,Dropout
from keras import utils
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

In [1]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(100, 100, 3),
    include_top=False)

In [2]:
base_model.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 100, 100, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 100, 100, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 50, 50, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 50, 50, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 50, 50, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 25, 25, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 25, 25, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 25, 25, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 25, 25, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 12, 12, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 12, 12, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 12, 12, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 6, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 3, 3, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:
base_model.trainable = False

In [4]:
inputs = keras.Input(shape=(100,100, 3))
# Separately from setting trainable on the model, we set training to False 
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(36)(x)
model = keras.Model(inputs, outputs)

In [5]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 3, 3, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 36)             │        18,468 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,733,156 (56.20 MB)

 Trainable params: 18,468 (72.14 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [7]:
model.compile(loss=keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [72]:
Train_Directory=r"D:\python\ece\train"
Category=["apple","banana","beetroot","bell_pepper","cabbage","capsicum","carrot","cauliflower","chilli_pepper","corn","cucumber","eggplant","garlic","ginger","grapes","jalepeno","kiwi","lemon","lettuce","mango","onion","orange","paprika","pear","peas","pineapple","pomegranate","potato","raddish","soy_beans","sweetcorn","tomato","turnip","watermelon"]

In [73]:
Test_Directory=r"D:\python\ece\test"


In [74]:
Train_data=[]
for category in Category:
    folder=os.path.join(Train_Directory,category)
    label=Category.index(category)
    for img in os.listdir(folder):
        img_path=os.path.join(folder,img)
        img_arr=cv2.imread(img_path)
        if img_arr is None:
            continue
        else:
            img_arr=cv2.resize(img_arr,(100,100))
            Train_data.append([img_arr,label])
        

In [76]:
Valid_data=[]
for category in Category:
    folder=os.path.join(Test_Directory,category)
    label=Category.index(category)
    for img in os.listdir(folder):
        img_path=os.path.join(folder,img)
        img_arr=cv2.imread(img_path)
        if img_arr is None:
            continue
        else:
            img_arr=cv2.resize(img_arr,(100,100))
            Valid_data.append([img_arr,label])

In [77]:
x_train=[]
y_train=[]
for features,labels in Train_data:
    x_train.append(features)
    y_train.append(labels)

In [78]:
x_test=[]
y_test=[]
for features,labels in Valid_data:
    x_test.append(features)
    y_test.append(labels)

In [79]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [80]:
x_train=x_train/255
x_test=x_test/255


In [81]:
num_classes=36
y_train=keras.utils.to_categorical(y_train,num_classes)
y_test=keras.utils.to_categorical(y_test,num_classes)

In [82]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range=0.1,  # Randomly zoom image
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images horizontally
    vertical_flip=True, # Don't randomly flip images vertically
)  

In [83]:
batch_size = 32
img_iter = datagen.flow(x_train, y_train, batch_size=batch_size)

In [85]:
datagen.fit(x_train)

In [86]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [90]:
model.fit(img_iter,
          epochs=20,
          steps_per_epoch=int(len(x_train)/batch_size), # Run same number of steps we would if we were not using a generator.
          validation_data=(x_test, y_test))

Epoch 1/20


c:\Users\bmbar\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


92/92 ━━━━━━━━━━━━━━━━━━━━ 54s 562ms/step - accuracy: 0.0250 - loss: 8.5647 - val_accuracy: 0.0088 - val_loss: 10.9535
Epoch 2/20
 1/92 ━━━━━━━━━━━━━━━━━━━━ 51s 564ms/step - accuracy: 0.0000e+00 - loss: 11.6668

c:\Users\bmbar\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


92/92 ━━━━━━━━━━━━━━━━━━━━ 7s 70ms/step - accuracy: 0.0000e+00 - loss: 11.6668 - val_accuracy: 0.0088 - val_loss: 10.7155
Epoch 3/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 57s 615ms/step - accuracy: 0.0219 - loss: 9.3289 - val_accuracy: 0.0177 - val_loss: 10.5599
Epoch 4/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 6s 64ms/step - accuracy: 0.0625 - loss: 10.0802 - val_accuracy: 0.0177 - val_loss: 8.2365
Epoch 5/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 66s 712ms/step - accuracy: 0.0214 - loss: 9.6803 - val_accuracy: 0.0147 - val_loss: 10.7959
Epoch 6/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 7s 66ms/step - accuracy: 0.0000e+00 - loss: 11.0862 - val_accuracy: 0.0147 - val_loss: 10.9855
Epoch 7/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 58s 632ms/step - accuracy: 0.0220 - loss: 9.6890 - val_accuracy: 0.0295 - val_loss: 10.2289
Epoch 8/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 6s 65ms/step - accuracy: 0.0312 - loss: 9.5825 - val_accuracy: 0.0295 - val_loss: 11.3641
Epoch 9/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 57s 619ms/step - accuracy: 0.0284 - loss: 9.8064 - val_accura

In [91]:
# Unfreeze the base model
base_model.trainable = True

# It's important to recompile your model after you make any changes
# to the `trainable` attribute of any inner layer, so that your changes
# are taken into account
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .001),  # Very low learning rate
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [111]:
model.fit(img_iter,steps_per_epoch=92, validation_data=(x_test, y_test), epochs=32)

Epoch 1/32
92/92 ━━━━━━━━━━━━━━━━━━━━ 123s 1s/step - accuracy: 0.2867 - loss: 2.2849 - val_accuracy: 0.4071 - val_loss: 1.8950
Epoch 2/32
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 44ms/step - accuracy: 0.2188 - loss: 2.3379 - val_accuracy: 0.3894 - val_loss: 1.8246
Epoch 3/32
92/92 ━━━━━━━━━━━━━━━━━━━━ 125s 1s/step - accuracy: 0.3123 - loss: 2.2510 - val_accuracy: 0.4897 - val_loss: 1.7009
Epoch 4/32
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 44ms/step - accuracy: 0.3438 - loss: 2.1935 - val_accuracy: 0.4454 - val_loss: 1.7669
Epoch 5/32
92/92 ━━━━━━━━━━━━━━━━━━━━ 126s 1s/step - accuracy: 0.3064 - loss: 2.2242 - val_accuracy: 0.4572 - val_loss: 1.6969
Epoch 6/32
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 44ms/step - accuracy: 0.5312 - loss: 1.9354 - val_accuracy: 0.4454 - val_loss: 1.7169
Epoch 7/32
92/92 ━━━━━━━━━━━━━━━━━━━━ 126s 1s/step - accuracy: 0.3426 - loss: 2.1591 - val_accuracy: 0.4130 - val_loss: 1.7498
Epoch 8/32
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - accuracy: 0.2500 - loss: 2.1385 - val_accuracy: 0.5074 - v

In [112]:
model.save("fruit_detector2.h5")